In [ ]:
###########################################################
# Imports
###########################################################
import sys
import os
sys.path.insert(0, os.getcwd()[0:len(os.getcwd()) - 11])
sys.path.insert(1, os.getcwd()[0:len(os.getcwd()) - 11] + '/Metrics')

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import sys
import itertools

import Models
import TrainingMetrics

In [ ]:
###########################################################
# Define file
###########################################################

trainFileName = sys.path[0] + '/files/hierarchy_TRAIN_shower.npz'

classifierModelPath = sys.path[0] + '/models/primary_shower_classifier_model'

In [ ]:
###########################################################
# Hyperparameters
###########################################################

N_EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
DROPOUT_RATE = 0.5

In [ ]:
###########################################################
# Get data from file
###########################################################

data = np.load(trainFileName)

# Variables
variables_train = data['variables_train']
variables_test = data['variables_test']
# Training cut
trainingCutDCA_train = data['trainingCutDCA_train']
trainingCutDCA_test = data['trainingCutDCA_test']
# Truth
isTruePrimaryLink_train = data['isTruePrimaryLink_train']
isTruePrimaryLink_test = data['isTruePrimaryLink_test']

In [ ]:
###########################################################
# Check shapes
###########################################################

print('variables_train.shape:', variables_train.shape)
print('variables_test.shape:', variables_test.shape)
print('trainingCutDCA_train.shape:', trainingCutDCA_train.shape)
print('trainingCutDCA_test.shape:', trainingCutDCA_test.shape)
print('isTruePrimaryLink_train.shape:', isTruePrimaryLink_train.shape)
print('isTruePrimaryLink_test.shape:', isTruePrimaryLink_test.shape)

nVariables = variables_train.shape[1]
ntrain = variables_train.shape[0]
ntest  = variables_test.shape[0]

In [ ]:
###########################################################
# Apply training cut topology mask
###########################################################

# These are inspired by hand-scanning :'(
MAX_TRAINING_CUT_DCA = 50.0

######################
# training set first
######################
# Make mask
passTrainingCutDCA_train = trainingCutDCA_train < MAX_TRAINING_CUT_DCA
passTrainingCuts_train = passTrainingCutDCA_train

# Mask the 1D variables... shape=(nEntries, )
trainingCutDCA_train = trainingCutDCA_train[passTrainingCuts_train]
isTruePrimaryLink_train = isTruePrimaryLink_train[passTrainingCuts_train]

# Mask the variable... shape=(nEntries, nVariables)
variables_train = variables_train[[[entry] * nVariables for entry in passTrainingCuts_train]].reshape(-1, nVariables)

######################
# now test set
######################
# Make mask
passTrainingCutDCA_test = trainingCutDCA_test < MAX_TRAINING_CUT_DCA
passTrainingCuts_test = passTrainingCutDCA_test

# Mask the 1D variables... shape=(nEntries, )
trainingCutDCA_test = trainingCutDCA_test[passTrainingCuts_test]
isTruePrimaryLink_test = isTruePrimaryLink_test[passTrainingCuts_test]

# Mask the variable... shape=(nEntries, nVariables)
variables_test = variables_test[[[entry] * nVariables for entry in passTrainingCuts_test]].reshape(-1, nVariables)

In [ ]:
###########################################################
# Check shapes
###########################################################

print('variables_train.shape:', variables_train.shape)
print('variables_test.shape:', variables_test.shape)
print('trainingCutDCA_train.shape:', trainingCutDCA_train.shape)
print('trainingCutDCA_test.shape:', trainingCutDCA_test.shape)
print('isTruePrimaryLink_train.shape:', isTruePrimaryLink_train.shape)
print('isTruePrimaryLink_test.shape:', isTruePrimaryLink_test.shape)

nVariables = variables_train.shape[1]
ntrain = variables_train.shape[0]
ntest  = variables_test.shape[0]

print('')
print('ntrain:', ntrain)
print('ntest:', ntest)

In [ ]:
###########################################################
# Prepare Dataset objects
###########################################################

loader_train = DataLoader(list(zip(variables_train, isTruePrimaryLink_train)), shuffle=True, batch_size=BATCH_SIZE)
loader_test = DataLoader(list(zip(variables_test, isTruePrimaryLink_test)), shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
###########################################################
# Define model and optimiser and compile the model
###########################################################

model = Models.PrimaryTrackShowerModel(nVariables, dropoutRate=DROPOUT_RATE)

In [ ]:
###########################################################
# Define class weights
###########################################################

nTrue_final = np.count_nonzero(isTruePrimaryLink_train == 1)
nBackground_final = np.count_nonzero(isTruePrimaryLink_train == 0)

maxLinks = max(nTrue_final, nBackground_final)

classWeights_final = {0: maxLinks/nBackground_final, 1: maxLinks/nTrue_final}

print('nTrue_final:', nTrue_final)
print('nBackground_final:', nBackground_final)
print('class weights:', classWeights_final)

In [ ]:
######################
# Training and testing
######################

# Optimiser
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

loss_func = torch.nn.BCELoss()
#loss_func = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights)

# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', factor=0.1, patience=3, threshold=0.0001, threshold_mode='abs')

# Put here some metrics
training_epoch = []
training_loss = []
training_accuracy = []
training_positive_as_positive_rate = []
training_positive_as_negative_rate = []
training_negative_as_negative_rate = []
training_negative_as_positive_rate = []

testing_epoch = []
testing_loss = []
testing_accuracy = []
testing_positive_as_positive_rate = []
testing_positive_as_negative_rate = []
testing_negative_as_negative_rate = []
testing_negative_as_positive_rate = []

for epoch in range(N_EPOCHS):
    
    ######################
    # Training 
    ######################
    
    model.train()
    
    # Iterate in batches over the training dataset.                        
    for inputs, targets in loader_train:  
        
        # Skip incomplete batches
        if (inputs.shape[0] != BATCH_SIZE) :
            continue        
            
        # Get predictions
        pred = model(inputs).reshape(-1)

        # Calc loss
        weight = torch.ones(targets.shape)
        weight[targets > 0.5] = classWeights_final[1]
        weight[targets < 0.5] = classWeights_final[0]
        loss_func.weight = weight
        targets = targets.to(torch.float64)
        pred = pred.type(torch.float64)
        loss = loss_func(pred, targets)
        
        # Update model parameters
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()   

    # Update the scheduler
    before_lr = optimiser.param_groups[0]["lr"]
    scheduler.step(loss)
    after_lr = optimiser.param_groups[0]["lr"]
        
    print("Epoch %d: SGD lr %.6f -> %.6f" % (epoch, before_lr, after_lr))
        
    ######################
    # Validation metrics 
    ######################
    with torch.no_grad():
        
        # Begin testing mode
        model.eval()
        
        # Initialise metrics        
        total_batches_train = 0
        total_loss_train = 0
        pos_scores_train = []
        neg_scores_train = []
                
        total_batches_test = 0
        total_loss_test = 0
        pos_scores_test = []
        neg_scores_test = []
                
        # Iterate in batches over the training dataset.                        
        for inputs_train, targets_train in loader_train:  

            # Skip incomplete batches
            if (inputs_train.shape[0] != BATCH_SIZE) :
                continue        

            # Get predictions
            pred_train = model(inputs_train).reshape(-1)

            # Calc loss
            targets_train = targets_train.to(torch.float64)
            pred_train = pred_train.type(torch.float64)
            loss_train = loss_func(pred_train, targets_train)

            # Add to our metrics
            total_batches_train += 1
            pos_scores_train.extend(np.array(pred_train.tolist())[targets_train > 0.5]) # they're not int, so IDK if == 1 works?
            neg_scores_train.extend(np.array(pred_train.tolist())[targets_train < 0.5]) # they're not int, so IDK if == 1 works?
            total_loss_train += loss_train.item()
            
        training_epoch.append(epoch)
            
        # Iterate in batches over the testing dataset.                        
        for inputs_test, targets_test in loader_test:  

            # Skip incomplete batches
            if (inputs_test.shape[0] != BATCH_SIZE) :
                continue        

            # Get predictions
            pred_test = model(inputs_test).reshape(-1)

            # Calc loss
            targets_test = targets_test.to(torch.float64)
            pred_test = pred_test.type(torch.float64)
            loss_test = loss_func(pred_test, targets_test)

            # Add to our metrics
            total_batches_test += 1
            pos_scores_test.extend(np.array(pred_test.tolist())[targets_test > 0.5]) # they're not int, so IDK if == 1 works?
            neg_scores_test.extend(np.array(pred_test.tolist())[targets_test < 0.5]) # they're not int, so IDK if == 1 works?
            total_loss_test += loss_test.item()      
            
        testing_epoch.append(epoch)
    
    ##########################
    # Calc metrics for epoch 
    ##########################   
    # train
    optimal_threshold_train, maximum_accuracy_train = TrainingMetrics.calculate_accuracy(torch.tensor(pos_scores_train), torch.tensor(neg_scores_train)) 
    # test
    optimal_threshold_test, maximum_accuracy_test = TrainingMetrics.calculate_accuracy(torch.tensor(pos_scores_test), torch.tensor(neg_scores_test))

    # train
    positive_as_positive_train = np.count_nonzero(np.array(pos_scores_train) > optimal_threshold_train)
    positive_as_negative_train = np.count_nonzero(np.array(pos_scores_train) < optimal_threshold_train)
    negative_as_positive_train = np.count_nonzero(np.array(neg_scores_train) > optimal_threshold_train)
    negative_as_negative_train = np.count_nonzero(np.array(neg_scores_train) < optimal_threshold_train)
    # test
    positive_as_positive_test = np.count_nonzero(np.array(pos_scores_test) > optimal_threshold_test)
    positive_as_negative_test = np.count_nonzero(np.array(pos_scores_test) < optimal_threshold_test)
    negative_as_positive_test = np.count_nonzero(np.array(neg_scores_test) > optimal_threshold_test)
    negative_as_negative_test = np.count_nonzero(np.array(neg_scores_test) < optimal_threshold_test)
    
    # train
    positive_as_positive_fraction_train = float(positive_as_positive_train) / float(positive_as_positive_train + positive_as_negative_train)
    positive_as_negative_fraction_train = float(positive_as_negative_train) / float(positive_as_positive_train + positive_as_negative_train)
    negative_as_positive_fraction_train = float(negative_as_positive_train) / float(negative_as_positive_train + negative_as_negative_train)
    negative_as_negative_fraction_train = float(negative_as_negative_train) / float(negative_as_positive_train + negative_as_negative_train)
    # test
    positive_as_positive_fraction_test = float(positive_as_positive_test) / float(positive_as_positive_test + positive_as_negative_test)
    positive_as_negative_fraction_test = float(positive_as_negative_test) / float(positive_as_positive_test + positive_as_negative_test)
    negative_as_positive_fraction_test = float(negative_as_positive_test) / float(negative_as_positive_test + negative_as_negative_test)
    negative_as_negative_fraction_test = float(negative_as_negative_test) / float(negative_as_positive_test + negative_as_negative_test)
    
    # train
    training_loss.append(float(total_loss_train) / float(total_batches_train))
    training_accuracy.append(maximum_accuracy_train)
    training_positive_as_positive_rate.append(positive_as_positive_fraction_train)
    training_positive_as_negative_rate.append(positive_as_negative_fraction_train)
    training_negative_as_negative_rate.append(negative_as_negative_fraction_train)
    training_negative_as_positive_rate.append(negative_as_positive_fraction_train)
    # test
    testing_loss.append(float(total_loss_test) / float(total_batches_test))
    testing_accuracy.append(maximum_accuracy_test)
    testing_positive_as_positive_rate.append(positive_as_positive_fraction_test)
    testing_positive_as_negative_rate.append(positive_as_negative_fraction_test)
    testing_negative_as_negative_rate.append(negative_as_negative_fraction_test)
    testing_negative_as_positive_rate.append(negative_as_positive_fraction_test)

    # Do some prints
    print('----------------------------------------')
    print('Epoch:', epoch)
    print('----------------------------------------')
    print('loss_train:', round(training_loss[-1], 2))
    print('----')
    print('optimal_threshold_train:', optimal_threshold_train)
    print('accuracy_train:', str(round(maximum_accuracy_train.item(), 2)) +'%')
    print('positive_as_positive_fraction_train:', str(round(positive_as_positive_fraction_train * 100.0, 2)) + '%')
    print('positive_as_negative_fraction_train:', str(round(positive_as_negative_fraction_train * 100.0, 2)) + '%')
    print('negative_as_negative_fraction_train:', str(round(negative_as_negative_fraction_train * 100.0, 2)) + '%')
    print('negative_as_positive_fraction_train:', str(round(negative_as_positive_fraction_train * 100.0, 2)) + '%')
    print('----')
    print('loss_test:', round(testing_loss[-1], 2))
    print('----')
    print('optimal_threshold_test:', optimal_threshold_test)
    print('accuracy_test:', str(round(maximum_accuracy_test.item(), 2)) +'%')
    print('positive_as_positive_fraction_test:', str(round(positive_as_positive_fraction_test * 100.0, 2)) + '%')
    print('positive_as_negative_fraction_test:', str(round(positive_as_negative_fraction_test * 100.0, 2)) + '%')
    print('negative_as_negative_fraction_test:', str(round(negative_as_negative_fraction_test * 100.0, 2)) + '%')
    print('negative_as_positive_fraction_test:', str(round(negative_as_positive_fraction_test * 100.0, 2)) + '%')
    print('----')
    TrainingMetrics.plot_scores_classifier(pos_scores_train, neg_scores_train, pos_scores_test, neg_scores_test)

In [ ]:
# Print metrics showing evolution   
TrainingMetrics.plot_loss_evolution(training_epoch, training_loss, testing_loss, 'Classifier')
TrainingMetrics.plot_edge_rate(training_epoch, training_positive_as_positive_rate, training_positive_as_negative_rate, testing_positive_as_positive_rate, testing_positive_as_negative_rate, True)
TrainingMetrics.plot_edge_rate(testing_epoch, training_negative_as_negative_rate, training_negative_as_positive_rate, testing_negative_as_negative_rate, testing_negative_as_positive_rate, False)

In [ ]:
######################
# Confusion matrices!
######################
with torch.no_grad():

    # Begin testing mode
    model.eval()
    
    pred_final_test = model(torch.tensor(variables_test))
    
    pos_scores_final_test = np.array(pred_final_test.tolist())[isTruePrimaryLink_test > 0.5] # they're not int, so IDK if == 1 works?
    neg_scores_final_test = np.array(pred_final_test.tolist())[isTruePrimaryLink_test < 0.5] # they're not int, so IDK if == 1 works?
    
    TrainingMetrics.plot_roc_curve(torch.tensor(pos_scores_final_test), torch.tensor(neg_scores_final_test))
    TrainingMetrics.draw_confusion_with_threshold(pred_final_test, isTruePrimaryLink_test, 0.5)
    TrainingMetrics.draw_confusion_with_threshold(pred_final_test, isTruePrimaryLink_test, 0.3)

In [ ]:
######################
# Save the model
###################### 

sm = torch.jit.script(model)
sm.save(classifierModelPath)